# **Zeke Morotn Homework 3**

---


# ***Which Fruit Is It?***

NOTE: The deadline is in 10 days!

The starter notebook for this homework is hw3_starter.ipynb. You must use the output already provided in the notebook, because we will be grading automatically.

In this homework your goal is to identify fruit based on the following attributes: mass, width, height and color. The data set is slightly permutated  and is available here: https://raw.githubusercontent.com/lkyin/ECS189L/main/fruits.csv

Specifically, your goal is to:

1. Build a logistic regression classifier to predict the fruit_name variable in terms of the predictors mass, width, height, and color_score. Make sure the model is:
- the best one you can get (based on area under the ROC), and
- is not overfitted (use cross validation).
- ***What to report: a) the model coefficients, b) confusion matrix, and c) the area under ROC of your best model.***

2. Here, build a decision tree classifier to predict the fruit_name variable in terms of the predictors mass, width, height, and color_score (You are welcome to use regularization, but it is not required). Make sure the model is:
- the best one you can get (based on accuracy on test data),
- is not overfitted (use cross validation).
- ***What to report: a) plot of the decision tree and b) the accuracy of your best model on the test data.***

3. Draw the decision boundaries for the two models above and consider their performance vis-a-vis the decision boundaries. Answer the following question in a text cell. What to report:

- ***What do you notice in terms of the relationship between model performance and decision boundary for each model?***


##  ***Load the Data Set***

In [3]:
import pandas as pd

# load the data
df = pd.read_csv('data.csv', index_col=[0])

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)>

In [ ]:
# overview 
df.head(10)

## ***Q1. Build a logistic regression classifier to predict the fruit_name variable in terms of the predictors mass, width, height, and color_score. Make sure the model is:***
- the best one you can get (based on area under the ROC), and
- is not overfitted (use cross validation).
- ***What to report: a) the model coefficients, b) confusion matrix, and c) the area under ROC of your best model.***

### Select the dependent variable, and your independent variables of interest.

In [ ]:
# YOUR CODE
target = 'fruit_name'
Y = df[target]

# YOUR CODE
features = ['mass', 'width', 'height', 'color_score']
X = df[features]

### Split Training and Testing Using sklearn

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, 
                                                    random_state=42)


### Build your model(s) with traning X and Training Y

In [ ]:
from sklearn.model_selection import cross_validate as cv
from sklearn.linear_model import LogisticRegression as lr

# Found on StackOverflow to silence warning from sklearn. Link Bellow:
# https://stackoverflow.com/questions/32612180/eliminating-warnings-from-scikit-learn
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


model_log = lr(penalty='l2', C=0.5, solver='saga', multi_class='ovr',
           max_iter=5000, fit_intercept=True)
cv_results = cv(model_log, X_train, Y_train, cv=3)
print("The Average Score from Cross Validation is: {:.4f}".format(cv_results['test_score'].mean()))

clf = model_log.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
print("The Score from the best model on the test set is: {:.4f}".format(score))

### Show the summary of your best model(s)

In [ ]:
from sklearn.metrics import confusion_matrix

cols = clf.classes_
pred = pd.DataFrame(clf.predict(X_test))

print('Confusion Matrix: ')
confusion = pd.DataFrame(confusion_matrix(Y_test, pred), columns=cols)
confusion = confusion.add_prefix('Pred_')
confusion = confusion.rename(index={0: 'Actual_apple',1: 'Actual_lemon',
                        2: 'Actual_mandarin',3: 'Actual_orange' })
print(confusion)


print('\n\nLogistic Coeficients:')
coef = pd.DataFrame(clf.coef_, columns=X_test.columns)
coef = coef.rename(index={0: 'apple',1: 'lemon',
                        2: 'mandarin',3: 'orange' })
coef['intercept'] = clf.intercept_
print(coef)


### Plot the ROC curve(s) and show the AUC value(s).

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib  import pyplot as plt

actual = pd.get_dummies(Y_test)
cols = clf.classes_
pred = pd.DataFrame(clf.predict_proba(X_test))
pred.columns = cols

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')


for col in cols:
  fpr, tpr, thresholds = roc_curve(actual[col], pred[col])
  score = roc_auc_score(actual[col], pred[col])
  print('Area Under ROC for {} is {:.4f}'.format(col, score))
  plt.plot(fpr, tpr, label=col)
_ = plt.legend()
_ = plt.title("ROC for Logistic Classifier")
_ = plt.xlabel("Fale Positive Rate")
_ = plt.ylabel("True Positive Rate")


## ***Q2. Build a decision tree classifier to predict the fruit_name variable in terms of the predictors mass, width, height, and color_score. Make sure the model is:***
- the best one you can get (based on accuracy on test data),
- is not overfitted (use cross validation).
- ***What to report: a) plot of the decision tree and b) the accuracy of your best model on the test data.***



In [ ]:
from sklearn.tree import DecisionTreeClassifier as tree_class

model_tree = tree_class(random_state=42, ccp_alpha=0.025)
cv_results = cv(model_tree, X_train, Y_train, cv=3)

print("The Average Score from Cross Validation is: {:.4f}".format(cv_results['test_score'].mean()))

clf_tree = model_tree.fit(X_train, Y_train)
score = clf_tree.score(X_test, Y_test)
print("The Score from the best model on the test set is: {:.4f}".format(score))

In [ ]:
from sklearn.tree import plot_tree 

print('Plot of Decision Tree:')
plt.figure(figsize=(15,15))
plot_tree(clf_tree, filled=True, feature_names=X_test.columns, class_names=clf.classes_)
plt.show()

### ***Q3. Draw the decision boundaries for the two models above and consider their performance vis-a-vis the decision boundaries. Answer the following question in a text cell. What to report:***

- ***What do you notice in terms of the relationship between model performance and decision boundary for each model?***

In [ ]:
# used https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html as reference

import numpy as np

def dbound(x,y):

  # fit the models for the two features only
  clf_log = model_log.fit(X_train[[x,y]], Y_train)
  clf_tree = model_tree.fit(X_train[[x,y]], Y_train)


  # create meshgrid
  x_min, x_max = X[x].min()-0.25, X[x].max()+0.25
  y_min, y_max = X[y].min()-0.25, X[y].max()+0.25

  xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                       np.arange(y_min, y_max, 0.1))
  
  f, axarr = plt.subplots(1, 2, figsize=(15, 8))

  Z = clf_log.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = pd.Series(Z)
  Z = Z.map({'apple': 1, 'lemon': 2, 'orange': 3, 'mandarin':4})
  Z = Z.values
  Z = Z.reshape(xx.shape)


  axarr[0].contourf(xx, yy, Z, alpha=0.4)
  for fruit in ['apple', 'lemon', 'orange', 'mandarin']:
    axarr[0].scatter(X[Y==fruit][x], X[Y==fruit][y], s=20, edgecolor='k', label=fruit)
  axarr[0].set_title('Logistic Regresion for {} and {}'.format(x,y))
  axarr[0].legend()
  axarr[0].set_xlabel(x)
  axarr[0].set_ylabel(y)

  Z = clf_tree.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = pd.Series(Z)
  Z = Z.map({'apple': 1, 'lemon': 2, 'orange': 3, 'mandarin':4})
  Z = Z.values
  Z = Z.reshape(xx.shape)

  axarr[1].contourf(xx, yy, Z, alpha=0.4)
  for fruit in ['apple', 'lemon', 'orange', 'mandarin']:
    axarr[1].scatter(X[Y==fruit][x], X[Y==fruit][y], s=20, edgecolor='k', label=fruit)
  axarr[1].set_title('Tree Classifier for {} and {}'.format(x,y))
  axarr[1].legend()
  axarr[1].set_xlabel(x)
  axarr[1].set_ylabel(y)
  
  plt.show()


In [ ]:
# Decision Boundary for Logistic Regression and Decision Tree: 

# Feature Pair wise decision boundaries
for i in range(len(features)):
  for j in range(i + 1, len(features)):
    dbound(features[i],features[j])



***Report Here***

In general, if there are $m$ classes, logistic regression can use at most $m-1$ lines or hyperplanes to divide the feature space. These lines can have any slope, and if using polynomial regression, can also curve. 

In contrast, a tree classifier has one line or hyperplane per non leaf node to divide the feature space. These lines must be normal to one of the feature vectors, ie in two dimensions parralel to either $[0 ,1]$ or $[1,0]$

As we can see from the graph *Tree Classifier for width and height*, tree classifiers can separate bounderies in two separate areas to represent class that may appear in two distinct clusters in the feature space. This is something that logistic regression is unable to do. 

While with this data set, it does seem that tree classifier performs better in all graphs, it is possible to consider a data set where each class falls on parralel line with some non trivial slope in the feature space. In that case, logisitic regression would perform better than the tree classifier.

Neither model does very well when features are not linearly separable, resulting in a lot of misclassification. As you can see in the graphs for height and color score, both models do very well classifying the mandarins, but struggle with the rest. This is because the mandarins are linearly seperable from the rest of the fruit.

As far as model performance, it does look like in general, the decision tree does perform better compared to the logistic regrssion model. This must be because it can capture more complexity in where clusters of fruit are located. Likewise, model performance falls for both models when fruit clusters are intermixed in feature space.



---
###***Good Job! You are done here!***